# Exploratory Data Analysis and Data Prep in PySpark

In this workbook, you will read in the `trip` and `fare` files. You are welcome to use DataFrame and/or SparkSQL API as you desire as long as it produces the expected results.

It is recommended to make small versions of the datasets for testing purposes so that you can trial and error faster. Run a command similar to `df_small = df.limit(10000)`

Make sure you use your [PySparkSQL Cheat Sheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) to help you with the commands to complete the assignment.

Instructions:

1. Read in both datasets and conduct initial exploration. 
    - Determine the shape of the data, column names, data types
    - Check for the number of missing values on the critical fields of the datasets - pickup_datetime, dropoff_datetime, passenger_count, trip_distance, fare_amount, medallion, hack_license, etc. There is a `count` command to use with columns in a dataframe and several ways to check for missing values. Try Googling some options!
    - Check the counts for values of passenger_count and remove any outlier values (how many people fit into a taxi?). Justify your analytical decision. You can use the `.isin()` method to create boolean values from a column ([see here](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.Column.isin.html)).

2. Join both datasets such that you get a merged dataset with 21 unique fields. 
    - Which columns do you use to join the data? You need to determine how to match the schemas.
    - What type of join should you use? Try at least two types of joins and report how much data is matched and is lost based on the join type. Justify which join you selected.

3. Once you create the merged dataset, you need to convert fields to the following types, since all fields were read in as string:
    * pickup_datetime and dropoff_datetime must be TIMESTAMP
    * passenger_count and rate_code must be INT
    * all other numeric fields must be FLOAT
    * the remaining fields stay as STRING

4. Create new variables to your dataset that will be important for understanding the data.
    * Dummy variable for if pickup_datetime appears on a weekend or not
    * Dummy variable for if pickup_datetime appears during weekday rush hour (6:00-9:59am, 2:00-5:59pm) or not
    * Dummy variable for if the tip_amount is greater than 10% of the fare_amount or not

5. Save this merged, converted, and transformed dataset to your own S3 bucket in parquet format.

6. Report the average, median, Q25, and Q75 fare amount in a table conditioned on the dummy variables you created in step 4. Each table should look like the example below, though your numbers might not match exactly.


 |  weekdayrush_dummy  |mean_fare  |Q25|  median|   Q75|  num_rides|
|----------------------|-----------|---|--------|------|-----------|
|0               |True     |12.462  |6.5     |9.0  |14.0   |44848941|
|1              |False     |12.312  |6.5     |9.5  |14.0  |128336134|
       
You are welcome to add as many cells as you need. Clearly indicate in your notebook each step so graders can confirm you have accomplished each task. **You must include comments in your code.**

**REQUIRED:** Start off by copying the taxi data from the public S3 bucket to your personal S3 bucket!

In [1]:
!hadoop distcp s3://bigdatateaching/nyctaxi-2013/parquet/ s3://anly502-fall-2022-yl1353/taxi

2022-10-19 18:59:54,990 INFO tools.DistCp: Input Options: DistCpOptions{atomicCommit=false, syncFolder=false, deleteMissing=false, ignoreFailures=false, overwrite=false, append=false, useDiff=false, useRdiff=false, fromSnapshot=null, toSnapshot=null, skipCRC=false, blocking=true, numListstatusThreads=0, maxMaps=20, mapBandwidth=0.0, copyStrategy='uniformsize', preserveStatus=[BLOCKSIZE], atomicWorkPath=null, logPath=null, sourceFileListing=null, sourcePaths=[s3://bigdatateaching/nyctaxi-2013/parquet], targetPath=s3://anly502-fall-2022-yl1353/taxi, filtersFile='null', blocksPerChunk=0, copyBufferSize=8192, verboseLog=false, directWrite=false}, sourcePaths=[s3://bigdatateaching/nyctaxi-2013/parquet], targetPathExists=true, preserveRawXattrsfalse
2022-10-19 18:59:55,227 INFO client.RMProxy: Connecting to ResourceManager at ip-172-31-39-182.ec2.internal/172.31.39.182:8032
2022-10-19 18:59:55,436 INFO client.AHSProxy: Connecting to Application History server at ip-172-31-39-182.ec2.internal

In [2]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [3]:
sc = SparkContext()
spark = SparkSession.builder.appName("taxi-assignment").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/19 19:00:36 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
22/10/19 19:00:45 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


### Step 1: Read in data and explore

In [4]:
trip = spark.read.parquet("s3://anly502-fall-2022-yl1353/taxi/trip/")

In [5]:
trip.dtypes

[('medallion', 'string'),
 ('hack_license', 'string'),
 ('vendor_id', 'string'),
 ('rate_code', 'string'),
 ('store_and_fwd_flag', 'string'),
 ('pickup_datetime', 'string'),
 ('dropoff_datetime', 'string'),
 ('passenger_count', 'string'),
 ('trip_time_in_secs', 'string'),
 ('trip_distance', 'string'),
 ('pickup_longitude', 'string'),
 ('pickup_latitude', 'string'),
 ('dropoff_longitude', 'string'),
 ('dropoff_latitude', 'string')]

In [6]:
from pyspark.sql.functions import col,sum
trip.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in trip.columns)).show()

+---------+------------+---------+---------+------------------+---------------+----------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|medallion|hack_license|vendor_id|rate_code|store_and_fwd_flag|pickup_datetime|dropoff_datetime|passenger_count|trip_time_in_secs|trip_distance|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+---------+------------+---------+---------+------------------+---------------+----------------+---------------+-----------------+-------------+----------------+---------------+-----------------+----------------+
|        0|           0|        0|        0|          86519129|              0|               0|              0|                0|            0|               0|              0|             3438|            3438|
+---------+------------+---------+---------+------------------+---------------+----------------+---------------+-----------------+-------------+----

In [7]:
trip_c = trip.groupBy("passenger_count").count()
trip_c.show()

+---------------+---------+
|passenger_count|    count|
+---------------+---------+
|              7|       35|
|              9|       26|
|              3|  7315829|
|              0|     5035|
|            208|       13|
|            129|        1|
|              6|  6764789|
|              1|121959711|
|              8|       25|
|              5| 10034696|
|            255|        2|
|              4|  3582103|
|              2| 23517494|
+---------------+---------+



In [9]:
#remove outliers: 208,129,255
trip_p = trip_c[trip_c.passenger_count.isin([0,1,2,3,4,5,6])]

In [10]:
trip_p.show()

+---------------+---------+
|passenger_count|    count|
+---------------+---------+
|              3|  7315829|
|              0|     5035|
|              6|  6764789|
|              1|121959711|
|              5| 10034696|
|              4|  3582103|
|              2| 23517494|
+---------------+---------+



In [11]:
fare = spark.read.parquet("s3://anly502-fall-2022-yl1353/taxi/fare/")

In [12]:
fare.dtypes

[('medallion', 'string'),
 ('hack_license', 'string'),
 ('vendor_id', 'string'),
 ('pickup_datetime', 'string'),
 ('payment_type', 'string'),
 ('fare_amount', 'string'),
 ('surcharge', 'string'),
 ('mta_tax', 'string'),
 ('tip_amount', 'string'),
 ('tolls_amount', 'string'),
 ('total_amount', 'string')]

In [13]:
fare.select(*(sum(col(c).isNull().cast("int")).alias(c) for c in fare.columns)).show()

+---------+------------+---------+---------------+------------+-----------+---------+-------+----------+------------+------------+
|medallion|hack_license|vendor_id|pickup_datetime|payment_type|fare_amount|surcharge|mta_tax|tip_amount|tolls_amount|total_amount|
+---------+------------+---------+---------------+------------+-----------+---------+-------+----------+------------+------------+
|        0|           0|        0|              0|           0|          0|        0|      0|         0|           0|           0|
+---------+------------+---------+---------------+------------+-----------+---------+-------+----------+------------+------------+



### Step 2: Join the datasets

In [14]:
merge_df_inner = trip.join(fare,['medallion','hack_license','vendor_id','pickup_datetime'],'inner').cache()
#merge_df = pd.merge(trip, fare, on='medallion')#,'hack_license', 'vendor_id','pickup_datetime'])

In [15]:
merge_df_outer = trip.join(fare,['medallion','hack_license','vendor_id','pickup_datetime'],"outer").cache()

In [18]:
print('row_count_inner:', merge_df_inner.count())
print('col_count_inner:', len(merge_df_inner.columns))

row_count_inner: 173185091
col_count_inner: 21


lost approx. 16k from original size of dataframes

In [20]:
print('row_count_outer:', merge_df_outer.count())
print('col_count_outer:', len(merge_df_outer.columns))

row_count_outer: 173185091
col_count_outer: 21


lost approx. 16k from original size of dataframes

I think the inner join makes more sense here, but I am a little confused that the final dataset generated from inner join and outer join have the same numbers of rows and columns

### Step 3: Change data types
This step can also be integrated into Step 2 merging if using PySparkSQL

In [21]:
from pyspark.sql.functions import *

merge_df_inner = merge_df_inner.withColumn("pickup_datetime",to_timestamp("pickup_datetime")) 
df = merge_df_inner.withColumn("dropoff_datetime",to_timestamp("dropoff_datetime")) 

In [22]:
from pyspark.sql.types import IntegerType

df = df.withColumn("passenger_count", df["passenger_count"].cast(IntegerType()))
df = df.withColumn("rate_code", df["rate_code"].cast(IntegerType()))

In [23]:
df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: string (nullable = true)
 |-- trip_distance: string (nullable = true)
 |-- pickup_longitude: string (nullable = true)
 |-- pickup_latitude: string (nullable = true)
 |-- dropoff_longitude: string (nullable = true)
 |-- dropoff_latitude: string (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: string (nullable = true)
 |-- surcharge: string (nullable = true)
 |-- mta_tax: string (nullable = true)
 |-- tip_amount: string (nullable = true)
 |-- tolls_amount: string (nullable = true)
 |-- total_amount: string (nullable = true)



In [24]:
num_col = ['trip_time_in_secs', 'trip_distance', 'pickup_longitude', 'pickup_latitude',
             'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'surcharge', 'mta_tax',
             'tip_amount',  'tolls_amount', 'total_amount']
for cols in num_col:
    df = df.withColumn(cols, col(cols).cast('float'))

In [25]:
df.printSchema()

root
 |-- medallion: string (nullable = true)
 |-- hack_license: string (nullable = true)
 |-- vendor_id: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_time_in_secs: float (nullable = true)
 |-- trip_distance: float (nullable = true)
 |-- pickup_longitude: float (nullable = true)
 |-- pickup_latitude: float (nullable = true)
 |-- dropoff_longitude: float (nullable = true)
 |-- dropoff_latitude: float (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- fare_amount: float (nullable = true)
 |-- surcharge: float (nullable = true)
 |-- mta_tax: float (nullable = true)
 |-- tip_amount: float (nullable = true)
 |-- tolls_amount: float (nullable = true)
 |-- total_amount: float (nullable = true)



### Step 4: Create dummy variables

- Dummy variable for if pickup_datetime appears on a weekend or not
- Dummy variable for if pickup_datetime appears during weekday rush hour (6:00-9:59am, 2:00-5:59pm) or not
- Dummy variable for if the tip_amount is greater than 10% of the fare_amount or not

Reminder that using a small dataframe for testing will help you develop your analytics faster.

In [26]:
from pyspark.sql.functions import dayofweek
df = df.withColumn("pickup_weekend_dummy", dayofweek("pickup_datetime").isin([1,7]).cast("int"))

In [27]:
df = df.withColumn('time', date_format('pickup_datetime', 'HH:mm:ss'))

In [28]:
df = df.withColumn("weekdayrush_dummy",
                   when((col("pickup_weekend_dummy") == 0) & 
                        ((col("time").between('6:00:00','9:59:00'))|
                         (col("time").between('14:00:00','17:59:00'))),1)
                   .otherwise(0))

In [29]:
df = df.withColumn('tip_amount_dummy', when(col('tip_amount') > col('fare_amount')*0.1, 1).otherwise(0))

In [30]:
df.take(3)

[Row(medallion='00005007A9F30E289E760362F69E4EAD', hack_license='02015F5B7D18846209A4DEDF896F6D9C', vendor_id='CMT', pickup_datetime=datetime.datetime(2013, 5, 20, 2, 15, 5), rate_code=1, store_and_fwd_flag='N', dropoff_datetime=datetime.datetime(2013, 5, 20, 2, 28, 51), passenger_count=2, trip_time_in_secs=827.0, trip_distance=5.300000190734863, pickup_longitude=-74.0083999633789, pickup_latitude=40.740516662597656, dropoff_longitude=-73.96519470214844, dropoff_latitude=40.801292419433594, payment_type='CRD', fare_amount=17.5, surcharge=0.5, mta_tax=0.5, tip_amount=3.700000047683716, tolls_amount=0.0, total_amount=22.200000762939453, pickup_weekend_dummy=0, time='02:15:05', weekdayrush_dummy=0, tip_amount_dummy=1),
 Row(medallion='00005007A9F30E289E760362F69E4EAD', hack_license='0649DA10C83DE7C6A1CD6BDC97DAB068', vendor_id='CMT', pickup_datetime=datetime.datetime(2013, 5, 14, 20, 28, 38), rate_code=1, store_and_fwd_flag='N', dropoff_datetime=datetime.datetime(2013, 5, 14, 20, 37, 35),

### Step 5: Save data to S3

In [37]:
df.write.parquet("s3://anly502-fall-2022-yl1353/taxi/a6-part2/")

### Step 6: Analytics

In [38]:
#table for weekend
df_weekend = (df.groupBy((when(col("pickup_weekend_dummy") == 1, 'True').
                          otherwise('False')).alias('pickup_weekend_dummy')) #groupby by weekend or not
              
              #Report the average, median, Q25, and Q75 fare amount
              .agg(mean('fare_amount').alias('mean_fare'),
                   expr('percentile(fare_amount, array(0.25))')[0].alias('Q25'),
                   expr('percentile(fare_amount, array(0.5))')[0].alias('median'),
                   expr('percentile(fare_amount, array(0.75))')[0].alias('Q75'), 
                   count(lit(1)).alias('num_rides'))
              .toPandas()) #convert to df

In [45]:
df_weekend

,pickup_weekend_dummy,mean_fare,Q25,median,Q75,num_rides
0,True,12.267475,6.5,9.5,14.0,49169382
1,False,12.383558,6.5,9.5,14.0,124015709


In [46]:
#table for rushhour
df_rushhour = (df.groupBy((when(col("weekdayrush_dummy") == 1, 'True').
                           otherwise('False')).alias('weekdayrush_dummy')) #groupby by weekday rushhour or not
               
               #Report the average, median, Q25, and Q75 fare amount
               .agg(mean('fare_amount').alias('mean_fare'),
                    expr('percentile(fare_amount, array(0.25))')[0].alias('Q25'),
                    expr('percentile(fare_amount, array(0.5))')[0].alias('median'),
                    expr('percentile(fare_amount, array(0.75))')[0].alias('Q75'),
                    count(lit(1)).alias('num_rides'))
               .toPandas()) #convert to df

In [47]:
df_rushhour

,weekdayrush_dummy,mean_fare,Q25,median,Q75,num_rides
0,True,12.995909,6.5,9.5,14.5,22901264
1,False,12.252265,6.5,9.5,14.0,150283827


In [48]:
df_tip = (df.groupBy((when(col("tip_amount_dummy") == 1, 'True').
                      otherwise('False')).alias('tip_amount_dummy')) #groupby by tip_amount is greater than 10% of the fare_amount or not
          
          #Report the average, median, Q25, and Q75 fare amount
          .agg(mean('fare_amount').alias('mean_fare'),
               expr('percentile(fare_amount, array(0.25))')[0].alias('Q25'),
               expr('percentile(fare_amount, array(0.5))')[0].alias('median'),
               expr('percentile(fare_amount, array(0.75))')[0].alias('Q75'),
               count(lit(1)).alias('num_rides'))
          .toPandas()) #convert to df

In [49]:
df_tip

,tip_amount_dummy,mean_fare,Q25,median,Q75,num_rides
0,True,12.766442,7.0,9.5,14.5,83788961
1,False,11.960842,6.5,9.0,13.5,89396130


## **Save your analytics results to a json object - then add, commit, and push your notebook and json to GitHub!**

In [50]:
import json
json.dump({'weekend' : df_weekend.to_dict('records'),
           'tip' : df_tip.to_dict('records'),
           'rushhour' : df_rushhour.to_dict('records')
          }, 
          fp = open('taxi-soln.json','w'))

# MAKE SURE YOU STOP YOUR EMR CLUSTER!

In [51]:
spark.stop()